In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [2]:
DATA_DIR = '/mnt/chicm/data/open-images/relation'

In [3]:
def get_top_classes(start_index=0, end_index=57):
    df = pd.read_csv(os.path.join(DATA_DIR, 'top_classes.csv'))
    #print(df.shape)
    c = df['class'].values[start_index:end_index]
    #print(df.head())
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi


In [4]:
classes, _ = get_top_classes()
classes[:5]

array(['/m/04yx4', '/m/03bt1vf', '/m/0k4j', '/m/05r655', '/m/01mzpv'],
      dtype=object)

In [5]:
classes_57 = set(classes)

In [6]:
def get_det(pred_str):
    dets = []
    det = []
    for i, e in enumerate(pred_str.split(' ')):
        if i % 6 == 0:
            det = []
        det.append(e)
        if (i+1) % 6 == 0:
            if det[0] in classes_57: #and float(det[1]) > 0.1:
                dets.append(det)
            
    return dets

In [7]:
!ls ../vrd/det -lh

total 3.2G
-rw-rw-r-- 1 core core 319M Sep 26 16:07 det_filter_57_lb61973_top200.csv
-rw-rw-r-- 1 core core 368M Sep 27 04:47 det_filter_57_lb61973_top400.csv
-rw-rw-r-- 1 core core 971M Sep 27 07:04 ensemble_chicm_od619_0927_top300.csv
-rw-rw-r-- 1 core core 911M Sep 26 15:53 final_ensemble_20190926_triple_high_thres.csv.zip
-rw-rw-r-- 1 core core 500M Sep 28 14:50 final_vrd_57_ensemble_20190928_5.csv
-rw-rw-r-- 1 core core 148M Sep 29 09:41 final_vrd_57_ensemble_20190928_5.csv.zip


In [9]:
df_det = pd.read_csv('../vrd/det/final_vrd_57_ensemble_20190928_5.csv')
#df_det = pd.read_csv('/mnt/chicm/open-images-vrd/vrd/lb25388/ensemble_detect_57_0820_1_top100.csv')

df_det.head()

,ImageId,PredictionString
0,00000b4dcff7f799,/m/04yx4 0.7008 0.8608 0.3073 0.9540 0.8078 /m...
1,00001a21632de752,/m/04yx4 0.6949 0.0076 0.4901 0.1846 0.9072 /m...
2,0000d67245642c5f,/m/05r5c 0.0130 0.1470 0.0110 1.0000 1.0000 /m...
3,0001244aa8ed3099,/m/0k4j 0.0457 0.2855 0.5084 0.4370 0.5919 /m/...
4,000172d1dd1adce0,/m/0bt9lr 0.0070 0.2530 0.0610 0.7850 0.8130 /...


In [10]:
df_det['dets'] = df_det.PredictionString.map(lambda x: get_det(str(x)))
df_det.head()

,ImageId,PredictionString,dets
0,00000b4dcff7f799,/m/04yx4 0.7008 0.8608 0.3073 0.9540 0.8078 /m...,"[[/m/04yx4, 0.7008, 0.8608, 0.3073, 0.9540, 0...."
1,00001a21632de752,/m/04yx4 0.6949 0.0076 0.4901 0.1846 0.9072 /m...,"[[/m/04yx4, 0.6949, 0.0076, 0.4901, 0.1846, 0...."
2,0000d67245642c5f,/m/05r5c 0.0130 0.1470 0.0110 1.0000 1.0000 /m...,"[[/m/05r5c, 0.0130, 0.1470, 0.0110, 1.0000, 1...."
3,0001244aa8ed3099,/m/0k4j 0.0457 0.2855 0.5084 0.4370 0.5919 /m/...,"[[/m/0k4j, 0.0457, 0.2855, 0.5084, 0.4370, 0.5..."
4,000172d1dd1adce0,/m/0bt9lr 0.0070 0.2530 0.0610 0.7850 0.8130 /...,"[[/m/0bt9lr, 0.0070, 0.2530, 0.0610, 0.7850, 0..."


In [11]:
df_det.dets.map(lambda x: len(x)).values.max()

1657

In [12]:
df_det.dets.map(lambda x: len(x)).values.sum()

11682665

In [13]:
df_det.dets.map(lambda x: len(x)).values.min()

0

In [14]:
def get_pred_str(dets):
    if len(dets) < 1:
        return ''
    res = []
    dets = sorted(dets, key=lambda x: x[1], reverse=True)[:200]
    for det in dets:
        res.extend(det)
        
    return ' '.join(res)

In [15]:
df_det['PredictionString'] = df_det.dets.map(lambda x: get_pred_str(x))
df_det.head()

,ImageId,PredictionString,dets
0,00000b4dcff7f799,/m/04yx4 0.7008 0.8608 0.3073 0.9540 0.8078 /m...,"[[/m/04yx4, 0.7008, 0.8608, 0.3073, 0.9540, 0...."
1,00001a21632de752,/m/04yx4 0.7176 0.6143 0.4868 0.7604 0.7347 /m...,"[[/m/04yx4, 0.6949, 0.0076, 0.4901, 0.1846, 0...."
2,0000d67245642c5f,/m/04ctx 0.0280 0.2441 0.2870 0.6494 0.8960 /m...,"[[/m/05r5c, 0.0130, 0.1470, 0.0110, 1.0000, 1...."
3,0001244aa8ed3099,/m/0k4j 0.0457 0.2855 0.5084 0.4370 0.5919 /m/...,"[[/m/0k4j, 0.0457, 0.2855, 0.5084, 0.4370, 0.5..."
4,000172d1dd1adce0,/m/04ctx 0.0230 0.3545 0.0469 0.8018 0.7683 /m...,"[[/m/0bt9lr, 0.0070, 0.2530, 0.0610, 0.7850, 0..."


In [ ]:
df_det.to_csv('../vrd/det/det_filter_57_gjx0929_top200.csv', columns=['ImageId', 'PredictionString'], index=False)

In [35]:
#!kaggle competitions submit -c open-images-2019-object-detection -f sub_filter_57_top150.csv -m "submit"

100%|████████████████████████████████████████| 297M/297M [00:18<00:00, 16.5MB/s]
Successfully submitted to Open Images 2019 - Object Detection